In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('combined_dataset_with_binary_labels.csv')
df

Audio_File                                      MFCC_Features  \
0    adrso024.mp3  [-368.61636    167.91826     18.401436    19.4...   
1    adrso025.mp3  [-466.84012    101.768555     1.5362022   24.1...   
2    adrso027.mp3  [-474.02808     99.658844    11.046471    24.5...   
3    adrso028.mp3  [-448.78406    117.59335     21.77527     13.4...   
4    adrso029.mp3  [-393.30402      93.440895      0.79102767   3...   
..            ...                                                ...   
232  adrso313.mp3  [-399.88904    128.72568     27.23024     14.2...   
233  adrso314.mp3  [-370.67444    128.5617      13.731075    17.1...   
234  adrso315.mp3  [-536.9598     105.8641      16.146296    18.6...   
235  adrso316.mp3  [-4.1812195e+02  1.3861430e+02  1.1947409e+01 ...   
236  adrso317.mp3  [-314.93112     132.62059      23.488382     2...   

         Label  Alzheimer_Status  
0    Alzheimer                 1  
1    Alzheimer                 1  
2    Alzheimer                 1  
3    Alzheimer                 1  
4    Alzheimer                 1  
..         ...               ...  
232    Control                 0  
233    Control                 0  
234    Control                 0  
235    Control                 0  
236    Control                 0  

[237 rows x 4 columns]

In [ ]:
# Iterate through the 'MFCC_Features' column in the DataFrame
for index, input_string in enumerate(df['MFCC_Features']):
    # Remove the square brackets
    input_string = input_string.strip('[]')

    # Split the string by whitespace and convert to floats
    float_list = [float(val) for val in input_string.split() if val.strip()]

    # Update the 'MFCC_Features' column with the list of floats for the current row
    df.at[index, 'MFCC_Features'] = float_list

df['MFCC_Features']

# Split the 'MFCC_Features' column into separate columns
df[['MFCC_Feature_1', 'MFCC_Feature_2', 'MFCC_Feature_3', 'MFCC_Feature_4',
    'MFCC_Feature_5', 'MFCC_Feature_6', 'MFCC_Feature_7', 'MFCC_Feature_8',
    'MFCC_Feature_9', 'MFCC_Feature_10', 'MFCC_Feature_11', 'MFCC_Feature_12',
    'MFCC_Feature_13']] = pd.DataFrame(df['MFCC_Features'].tolist(), index=df.index)

df;

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split data into features (X) and the target variable (y)
X = df[['MFCC_Feature_1', 'MFCC_Feature_2', 'MFCC_Feature_3', 'MFCC_Feature_4',
        'MFCC_Feature_5', 'MFCC_Feature_6', 'MFCC_Feature_7', 'MFCC_Feature_8',
        'MFCC_Feature_9', 'MFCC_Feature_10', 'MFCC_Feature_11', 'MFCC_Feature_12',
        'MFCC_Feature_13']]
# print(X)

y = df['Alzheimer_Status']
# print(y)

In [ ]:
# Split data into a training set and a testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from pickle import dump
from pickle import load
from sklearn.metrics import matthews_corrcoef

sc = StandardScaler()
X = sc.fit_transform(X)
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)

In [ ]:
param_grid = {'n_estimators': [100,150,200,250,300,350,400,500,700], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth' : [3,4,5,6,7,8,9,10], 'criterion' :['entropy', 'gini']}
model = RandomForestClassifier()


In [ ]:
loocv = LeaveOneOut()
scoring = 'accuracy'
kfold = KFold(n_splits=10, random_state=None)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

In [ ]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

Best: 0.656433 using {'criterion': 'entropy', 'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 100}


In [ ]:
best_model = grid_result.best_estimator_
best_model.fit(X_train, y_train)
filename = 'Random_Forest_Best.sav'
dump(best_model,open(filename, 'wb'))
loaded_model = load(open(filename, 'rb'))

In [ ]:
y_pred1 = loaded_model.predict(X_test)
MCC=matthews_corrcoef(y_test, y_pred1)

In [ ]:
cm1 = confusion_matrix(y_test, y_pred1)
acc = (cm1[0,0]+cm1[1,1])/(cm1[0,0]+cm1[0,1]+cm1[1,0]+cm1[1,1])
spec = (cm1[0,0])/(cm1[0,0]+cm1[0,1])
sens = (cm1[1,1])/(cm1[1,0]+cm1[1,1])

In [ ]:
print('Testing Accuracy =' ,acc)
print('Testing Sensitivity(abnormality) =' ,sens)
print('Testing Specificity(normality) =' ,spec)
print ('confusion matrix =', cm1)
print ('MCC Score =', MCC)
print(X.shape, y.shape)

Testing Accuracy = 0.5
Testing Sensitivity(abnormality) = 0.6538461538461539
Testing Specificity(normality) = 0.3181818181818182
confusion matrix = [[ 7 15]
 [ 9 17]]
MCC Score = -0.02956561979945413
(237, 13) (237,)


Support Vector Machines (SVM)



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from pickle import dump
from pickle import load
from sklearn.metrics import matthews_corrcoef

Run 1: test_size = 0.25

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

sc = StandardScaler()
X = sc.fit_transform(X)
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)


In [58]:
c_values = [10,50,100,300,500,700]
gammas=[1e-2,1e-1,1]
kernel_values = ['linear', 'poly', 'rbf', 'sigmoid']
param_grid = dict(C=c_values, gamma=gammas, kernel=kernel_values)

model = SVC()

In [59]:
loocv = LeaveOneOut()
scoring = 'accuracy'
kfold = KFold(n_splits=10, random_state=None)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, y_train)

In [60]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

Best: 0.710784 using {'C': 100, 'gamma': 0.01, 'kernel': 'rbf'}


In [61]:
best_model = grid_result.best_estimator_
best_model.fit(X_train, y_train)

SVC(C=100, gamma=0.01)

In [72]:
filename = 'SVM_default_split.sav'

dump(best_model,open(filename, 'wb'))
loaded_model = load(open(filename, 'rb'))

In [63]:
y_pred1 = loaded_model.predict(X_test)
MCC=matthews_corrcoef(y_test, y_pred1)

cm1 = confusion_matrix(y_test, y_pred1)
acc = (cm1[0,0]+cm1[1,1])/(cm1[0,0]+cm1[0,1]+cm1[1,0]+cm1[1,1])
spec = (cm1[0,0])/(cm1[0,0]+cm1[0,1])
sens = (cm1[1,1])/(cm1[1,0]+cm1[1,1])

print('Testing Accuracy =' ,acc)
print('Testing Sensitivity(abnormality) =' ,sens)
print('Testing Specificity(normality) =' ,spec)
print ('confusion matrix =', cm1)
print ('MCC Score =', MCC)
print(X.shape, y.shape)

Testing Accuracy = 0.7
Testing Sensitivity(abnormality) = 0.75
Testing Specificity(normality) = 0.65625
confusion matrix = [[21 11]
 [ 7 21]]
MCC Score = 0.40625
(237, 13) (237,)


Run 2 : test_size = 0.2, random_state = 0

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

sc = StandardScaler()
X = sc.fit_transform(X)
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)

In [66]:
c_values = [10,50,100,300,500,700]
gammas=[1e-2,1e-1,1]
kernel_values = ['linear', 'poly', 'rbf', 'sigmoid']
param_grid = dict(C=c_values, gamma=gammas, kernel=kernel_values)

model = SVC()

In [67]:
loocv = LeaveOneOut()
scoring = 'accuracy'
kfold = KFold(n_splits=10, random_state=None)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, y_train)

In [68]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

Best: 0.654971 using {'C': 500, 'gamma': 0.01, 'kernel': 'rbf'}


In [69]:
best_model = grid_result.best_estimator_
best_model.fit(X_train, y_train)

SVC(C=500, gamma=0.01)

In [70]:
filename = 'SVM_0.8_none.sav'

dump(best_model,open(filename, 'wb'))
loaded_model = load(open(filename, 'rb'))

In [71]:
y_pred1 = loaded_model.predict(X_test)
MCC=matthews_corrcoef(y_test, y_pred1)

cm1 = confusion_matrix(y_test, y_pred1)
acc = (cm1[0,0]+cm1[1,1])/(cm1[0,0]+cm1[0,1]+cm1[1,0]+cm1[1,1])
spec = (cm1[0,0])/(cm1[0,0]+cm1[0,1])
sens = (cm1[1,1])/(cm1[1,0]+cm1[1,1])

print('Testing Accuracy =' ,acc)
print('Testing Sensitivity(abnormality) =' ,sens)
print('Testing Specificity(normality) =' ,spec)
print ('confusion matrix =', cm1)
print ('MCC Score =', MCC)
print(X.shape, y.shape)

Testing Accuracy = 0.8125
Testing Sensitivity(abnormality) = 0.8333333333333334
Testing Specificity(normality) = 0.7916666666666666
confusion matrix = [[19  5]
 [ 4 20]]
MCC Score = 0.6255432421712243
(237, 13) (237,)


Run 3: test_size = 0.2, random_state = 42

In [73]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

sc = StandardScaler()
X = sc.fit_transform(X)
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
scaler = StandardScaler().fit(X_train)
rescaledX = scaler.transform(X_train)

In [74]:
c_values = [10,50,100,300,500,700]
gammas=[1e-2,1e-1,1]
kernel_values = ['linear', 'poly', 'rbf', 'sigmoid']
param_grid = dict(C=c_values, gamma=gammas, kernel=kernel_values)

model = SVC()

In [75]:
loocv = LeaveOneOut()
scoring = 'accuracy'
kfold = KFold(n_splits=10, random_state=None)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
grid_result = grid.fit(rescaledX, y_train)

In [76]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

Best: 0.745614 using {'C': 300, 'gamma': 0.01, 'kernel': 'rbf'}


In [77]:
best_model = grid_result.best_estimator_
best_model.fit(X_train, y_train)

SVC(C=300, gamma=0.01)

In [78]:
filename = 'SVM_0.8_42.sav'

dump(best_model,open(filename, 'wb'))
loaded_model = load(open(filename, 'rb'))

In [79]:
y_pred1 = loaded_model.predict(X_test)
MCC=matthews_corrcoef(y_test, y_pred1)

cm1 = confusion_matrix(y_test, y_pred1)
acc = (cm1[0,0]+cm1[1,1])/(cm1[0,0]+cm1[0,1]+cm1[1,0]+cm1[1,1])
spec = (cm1[0,0])/(cm1[0,0]+cm1[0,1])
sens = (cm1[1,1])/(cm1[1,0]+cm1[1,1])

print('Testing Accuracy =' ,acc)
print('Testing Sensitivity(abnormality) =' ,sens)
print('Testing Specificity(normality) =' ,spec)
print ('confusion matrix =', cm1)
print ('MCC Score =', MCC)
print(X.shape, y.shape)

Testing Accuracy = 0.6041666666666666
Testing Sensitivity(abnormality) = 0.6538461538461539
Testing Specificity(normality) = 0.5454545454545454
confusion matrix = [[12 10]
 [ 9 17]]
MCC Score = 0.20017752139298775
(237, 13) (237,)
